In [40]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Dense, Flatten, Dropout, MaxPool2D
from sklearn.model_selection import train_test_split
import pickle
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [61]:
! pip install kaggle

In [62]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [63]:
! cp kaggle.json ~/.kaggle/

In [64]:
! chmod 600 ~/.kaggle/kaggle.json

In [65]:
! kaggle datasets download jr2ngb/cataractdataset

cataractdataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [66]:
!unzip "/content/cataractdataset.zip"

Archive:  /content/cataractdataset.zip
replace README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [67]:
os.listdir('/content/dataset')

['2_cataract', '1_normal', '3_retina_disease', '2_glaucoma']

In [72]:
print('total normal eye images :', len(os.listdir('/content/dataset/1_normal')))
print('total cataract images :', len(os.listdir('/content/dataset/2_cataract')))
print('total glaucoma images :', len(os.listdir('/content/dataset/2_glaucoma')))
print('total retina disease images :', len(os.listdir('/content/dataset/3_retina_disease')))

total normal eye images : 300
total cataract images : 100
total glaucoma images : 101
total retina disease images : 100


In [73]:
train_dir = os.path.join('/content/dataset')
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode = 'nearest',
    validation_split=0.1) # set validation split
test_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range = 0.2,
    fill_mode = 'nearest',
    validation_split=0.4)

In [74]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical',
    subset='training')
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 541 images belonging to 4 classes.
Found 60 images belonging to 4 classes.


In [75]:
import tensorflow as tf
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4),  
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.4),  
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  
])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 32)        18464     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 36, 36, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 64)        1

In [76]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [77]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=50,
                    validation_steps=5,
                    verbose=2)

Epoch 1/50


InvalidArgumentError: ignored